# Algorithm Testing
A notebook used to compare algorithm performance.


In [1]:
### Imports
from nitools import utils 
from nitools.classifiers import PAE_RVFL, ELM, AE_ELM, PAE_ELM
from nitools.models import LeNet5, ResNet18, PeteNet
from nitools.operations import resetseed

import numpy as np
import random
import torch
import time as t

## 1. MNIST

Dataset of greyscale 28x28 images of handwritten digits. 

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*


In [2]:
# Import dataset
mnist = utils.load_mnist()

mnist_in = 784
mnist_class = 10

In [17]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_elm = PAE_ELM(
    360, 
    600, 
    mnist_class, 
    subnets=8,
    device=device, 
    c=10, ae_iters=3)

lenet = PeteNet(pae_elm)

X = torch.from_numpy(mnist['train_X']).float()[:30000]
y = torch.from_numpy(mnist['train_y']).float()[:30000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-ELM - MNIST', pred_arg, mnist['test_y'])

completed forward
torch.Size([30000, 10, 28, 28])
completed forward
torch.Size([30000, 10, 28, 28])
completed forward
torch.Size([30000, 10, 28, 28])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 14, 14])
completed forward
torch.Size([30000, 10, 6, 6])
completed forward
torch.Size([30000, 360])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed

/Users/peterdodd/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/peterdodd/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/peterdodd/miniforge3/envs/ml/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/peterdodd/miniforge3/e

### pAE-RVFL

In [14]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_rvfl = PAE_RVFL(
    360, 
    600, 
    mnist_class, 
    subnets=1,
    device=device, 
    r=(1,1), sc=0.5, sb=0.5, c=10, ae_iters=3)

lenet = PeteNet(pae_rvfl)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, mnist['test_y'])

completed forward
torch.Size([60000, 10, 28, 28])
completed forward
torch.Size([60000, 10, 28, 28])
completed forward
torch.Size([60000, 10, 28, 28])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 14, 14])
completed forward
torch.Size([60000, 10, 6, 6])
completed forward
torch.Size([60000, 360])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 28, 28])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed forward
torch.Size([10000, 10, 14, 14])
completed

### ELM

In [5]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = AE_ELM.AE_ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=100,
    output_size=mnist_class,
    ae_iters=3)

lenet = ResNet18.ResNet18(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:5000]
y = torch.from_numpy(mnist['train_y']).float()[:5000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet AE-ELM - MNIST', pred_arg, mnist['test_y'])

AttributeError: type object 'AE_ELM' has no attribute 'AE_ELM'

In [ ]:
SEED = 22
resetseed(SEED)

# Get ELM results
mnist_elm = ELM.ELM(  # TODO: batchnorm input weights to ae?
    input_size=mnist_in,
    h_size=1000,
    output_size=mnist_class)

lenet = LeNet5.LeNet5(mnist_elm, weight_train=True)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])


print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet  - MNIST', pred_arg, mnist['test_y'])

## 2. CIFAR-10

Dataset of colour (3-dimension) 32x32 images of objects of 10 classes:

0.	airplane
1.	automobile
2.	bird
3.	cat
4.	deer
5.	dog
6.	frog
7.	horse
8.	ship
9.	truck

*Train: 60,000*, *Test: 10,000*, *10 Classes*, *784 Inputs*

In [ ]:
# Import dataset
cifar10 = utils.load_cifar10()

cifar_in = 32*32*3
cifar_class = 10

### SPAE-RVFL

In [ ]:
device = torch.device("cpu")

pae_rvfl = SPAE_RVFL.SPAE_RVFL(
    cifar_in, 
    cifar_class, 
    [100], 
    subnets=[5], 
    device=device, 
    r=[(1,1), (1,1)], sc=0.5, sb=0.5)

lenet = LeNet5.LeNet5_3D(pae_rvfl)

X = torch.from_numpy(cifar10['train_X']).float()[:5000]
y = torch.from_numpy(cifar10['train_y']).float()[:5000]

result = lenet.train(X, y)
tX = torch.from_numpy(cifar10['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

utils.evaluation_summary('lenet PAE-RVFL - MNIST', pred_arg, cifar10['test_y'])

In [ ]:
# Show results

In [11]:
device = torch.device("cpu")

SEED = 22
resetseed(SEED)

pae_elm = PAE_ELM(
    360, 
    600, 
    mnist_class, 
    subnets=1,
    device=device, 
    c=10, ae_iters=3)

lenet = PeteNet(pae_elm)

X = torch.from_numpy(mnist['train_X']).float()[:60000]
y = torch.from_numpy(mnist['train_y']).float()[:60000]

train_start = t.time()
result = lenet.train(X, y)
train_end = t.time()
tX = torch.from_numpy(mnist['test_X']).float()
pred = lenet.predict(tX)[:10000]
pred_arg = np.zeros(10000)

for i in range(len(pred)):
    pred_arg[i] = np.argmax(pred[i])

print(f"train: {train_end-train_start}")
utils.evaluation_summary('lenet PAE-ELM - MNIST', pred_arg, mnist['test_y'])

completed forward
torch.Size([60000, 10, 28, 28])
completed forward
torch.Size([60000, 10, 28, 28])


KeyboardInterrupt: 